**Análise de Comércio Exterior - Base de Dados Detalhada por NCM**

Este notebook contém um código para análise de dados de comércio exterior utilizando a base de dados detalhada por NCM (Nomenclatura Comum do Mercosul). Os dados utilizados são obtidos a partir do link: [https://www.gov.br/produtividade-e-comercio-exterior/pt-br/assuntos/comercio-exterior/estatisticas/base-de-dados-bruta](https://www.gov.br/produtividade-e-comercio-exterior/pt-br/assuntos/comercio-exterior/estatisticas/base-de-dados-bruta).

O objetivo deste código é processar os arquivos CSV contendo dados de importação e exportação e realizar análises agregadas por estado, mês e NCM. As etapas principais do código são as seguintes:

1. Carregar os arquivos de importação e exportação em formato CSV.
2. Filtrar as colunas relevantes para cada arquivo.
3. Agrupar os dados de exportação por estado, mês e NCM.
4. Agrupar os dados de importação por estado, mês e NCM.
5. Obter a lista de estados únicos presentes nos dados.
6. Processar os dados para cada estado.
7. Filtrar os dados por estado e mês.
8. Combinar os dados de exportação e importação.
9. Calcular o valor líquido (exportação - importação).
10. Armazenar os dados processados em um dicionário.
11. Criar um DataFrame para cada estado com os dados de cada mês.
12. Calcular os somatórios anuais.
13. Salvar os resultados em arquivos CSV separados por estado.

Certifique-se de que os arquivos de importação e exportação estão presentes no local especificado pelos paths no código, substituindo "PATH_DO_ARQUIVO_IMP.csv" e "PATH_DO_ARQUIVO_EXP.csv" pelos caminhos corretos. O resultado da análise será salvo no diretório de saída especificado.

Execute o código sequencialmente para processar os dados e gerar as análises agregadas por estado, mês e NCM. Os resultados serão salvos em arquivos CSV individuais para cada estado.

Caso deseje utilizar outros arquivos de dados, certifique-se de que eles estejam no formato CSV e contenham as colunas necessárias para o processamento.

In [3]:
import pandas as pd
from collections import defaultdict

### Definir os paths dos arquivos de entrada e diretório de saída

In [ ]:
df_imp_path = "PATH_DO_ARQUIVO_IMP.csv"
df_exp_path = "PATH_DO_ARQUIVO_EXP.csv"
output_dir = "PATH_DO_DIRETORIO_DE_SAIDA"

### Carregar os arquivos .csv

In [4]:
df_imp = pd.read_csv(df_imp_path, delimiter=";", dtype={"CO_NCM": str})
df_exp = pd.read_csv(df_exp_path, delimiter=";", dtype={"CO_NCM": str})

### Filtrar as colunas relevantes para cada arquivo

In [5]:
df_imp = df_imp[["SG_UF_NCM", "CO_MES", "CO_NCM", "VL_FOB"]]
df_exp = df_exp[["SG_UF_NCM", "CO_MES", "CO_NCM", "VL_FOB"]]

### Agrupar dados de exportação por estado, mês e NCM

In [6]:
grouped_exp = df_exp.groupby(["SG_UF_NCM", "CO_MES", "CO_NCM"], as_index=False)["VL_FOB"].sum()

### Agrupar dados de importação por estado, mês e NCM

In [7]:
grouped_imp = df_imp.groupby(["SG_UF_NCM", "CO_MES", "CO_NCM"], as_index=False)["VL_FOB"].sum()

### Obter a lista de estados únicos

In [8]:
estados = set(df_imp["SG_UF_NCM"].unique()) | set(df_exp["SG_UF_NCM"].unique())

### Criar um dicionário para armazenar os dados de cada estado e mês

In [9]:
dados_estados = defaultdict(dict)

### Processar os dados para cada estado

In [10]:
for estado in estados:
    # Filtrar dados por estado
    df_exp_estado = grouped_exp[grouped_exp["SG_UF_NCM"] == estado]
    df_imp_estado = grouped_imp[grouped_imp["SG_UF_NCM"] == estado]

    # Obter a lista de meses únicos
    meses = set(df_imp_estado["CO_MES"].unique()) | set(df_exp_estado["CO_MES"].unique())

    # Processar os dados para cada mês
    for mes in meses:
        # Filtrar dados por mês
        df_exp_mes = df_exp_estado[df_exp_estado["CO_MES"] == mes]
        df_imp_mes = df_imp_estado[df_imp_estado["CO_MES"] == mes]

        # Combinar dados de exportação e importação
        df_mes = df_exp_mes.merge(df_imp_mes, on="CO_NCM", how="outer", suffixes=["_EXP", "_IMP"])

        # Calcular o valor líquido (exportação - importação)
        df_mes["VL_LIQUIDO"] = df_mes["VL_FOB_EXP"].fillna(0) - df_mes["VL_FOB_IMP"].fillna(0)

        # Obter os dados para o mês atual
        dados_mes_atual = df_mes[["CO_NCM", "VL_FOB_EXP", "VL_FOB_IMP", "VL_LIQUIDO"]]

        # Armazenar os dados do mês atual no dicionário de meses
        dados_estados[estado][mes] = dados_mes_atual

### Criar um DataFrame para cada estado com os dados de cada mês

In [11]:
for estado, dados_meses in dados_estados.items():
    df_estado = pd.DataFrame(columns=["CO_NCM"])
    for mes, dados_mes in dados_meses.items():
        df_mes = dados_mes.rename(columns={"VL_FOB_EXP": f"exp_{mes}", "VL_FOB_IMP": f"imp_{mes}", "VL_LIQUIDO": f"net_{mes}"})
        df_estado = df_estado.merge(df_mes, on="CO_NCM", how="outer")

    # Calcular os somatórios anuais
    df_estado["exp_total"] = df_estado.filter(like="exp_").sum(axis=1)
    df_estado["imp_total"] = df_estado.filter(like="imp_").sum(axis=1)
    df_estado["net_total"] = df_estado["exp_total"] - df_estado["imp_total"]

    # Salvar o DataFrame como arquivo CSV
    output_path = f"{output_dir}/{estado}.csv"
    df_estado.to_csv(output_path, index=False)